In [33]:
# Import dependencies

import pandas as pd
import gmaps
import requests

# Import API key

from config import g_key

In [34]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.dtypes

City_ID         int64
City           object
Country        object
Date           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
dtype: object

In [35]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [36]:
# Heatmap of temperature

# Get the latitude and longitude.

locations = city_data_df[["Lat", "Lng"]]

# Get the maximum temperature.

max_temp = city_data_df["Max Temp"]

# Assign the figure variable.

fig = gmaps.figure(center=(30.0,31.0), zoom_level=1.5)

# Assign the heatmap variable.

heat_layer = gmaps.heatmap_layer(locations, weights=[max(temp,0) for temp in max_temp], dissipating=False, max_intensity=300, point_radius=4)

# Add the heatmap layer.

fig.add_layer(heat_layer)

# Call the figure to plot the data.

fig

Figure(layout=FigureLayout(height='420px'))

In [37]:
# In the event a max temp was negative for 'heat_layer' use the following:


# # Get the maximum temperature.
# max_temp = city_data_df["Max Temp"]
# temps = []
# for temp in max_temp:
#     temps.append(max(temp, 0))

# using list comprehension the following code could be written for the for loop

# [max(temp, 0) for temp in max_temp]

# the heat_layer code would then look like 
# heat_layer = gmaps.heatmap_layer(locations, weights=[max(temp, 0) for temp in max_temp])

In [38]:
# Heatmap of percent humidity

locations = city_data_df[["Lat", "Lng"]]

humidity = city_data_df["Humidity"]

fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)

# Call the figure to plot the data.

fig

Figure(layout=FigureLayout(height='420px'))

In [39]:
# Heatmap of percent cloudiness

locations = city_data_df[["Lat", "Lng"]]

clouds = city_data_df["Cloudiness"]

fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

heat_layer = gmaps.heatmap_layer(locations, weights=clouds, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)

# Call the figure to plot the data.

fig

Figure(layout=FigureLayout(height='420px'))

In [40]:
# Heatmap of Wind Speed

locations = city_data_df[["Lat", "Lng"]]

wind = city_data_df["Wind Speed"]

fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

heat_layer = gmaps.heatmap_layer(locations, weights=wind, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)

# Call the figure to plot the data.

fig

Figure(layout=FigureLayout(height='420px'))

In [41]:
# Ask the customer to add a minimum and maximum temperature

min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [42]:
# Filter the dataset to find the cities that fit the criteria

preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]

preferred_cities_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
3,3,Harper,LR,2022-08-31 20:31:52,4.3750,-7.7169,75.15,91,100,12.21
5,5,Male,MV,2022-08-31 20:31:52,4.1748,73.5089,80.55,83,100,16.11
6,6,Torbay,CA,2022-08-31 20:31:53,47.6666,-52.7314,77.77,71,75,12.66
8,8,Tahta,EG,2022-08-31 20:31:54,26.7693,31.5021,88.11,34,0,13.91
11,11,Nishihara,JP,2022-08-31 20:31:55,26.1842,127.7558,83.48,87,75,20.71


In [43]:
preferred_cities_df.count()

City_ID       183
City          183
Country       183
Date          183
Lat           183
Lng           183
Max Temp      183
Humidity      183
Cloudiness    183
Wind Speed    183
dtype: int64

In [44]:
# Create a DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates

hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()

hotel_df["Hotel Name"] = ""

hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
3,Harper,LR,75.15,4.3750,-7.7169,
5,Male,MV,80.55,4.1748,73.5089,
6,Torbay,CA,77.77,47.6666,-52.7314,
8,Tahta,EG,88.11,26.7693,31.5021,
11,Nishihara,JP,83.48,26.1842,127.7558,
12,Albany,US,78.94,42.6001,-73.9662,
13,Thinadhoo,MV,80.92,0.5333,72.9333,
18,Port-Gentil,GA,76.96,-0.7193,8.7815,
20,Atuona,PF,77.09,-9.8000,-139.0333,
35,Eyl,SO,76.48,7.9803,49.8164,


In [45]:
# Set parameters to search for a hotel

params = {
    
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [46]:
# Iterate through the df

for index, row in hotel_df.iterrows():
    
    # Get the latitude and longitude
    
    lat = row["Lat"]
    
    lng = row["Lng"]
    
    # Add the latitude and longitude to location key for the params dictionary
    
    params["location"] = f"{lat},{lng}"
    
    # Use the search term: "lodging" and our latitude and longitude
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # MAke request and get the JSON data from the search.
    
    hotels = requests.get(base_url, params=params).json()
    
    # Grab the first hotel from the results and store the name.
    
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels['results'][0]['name']
        
    except (IndexError):
        
        print("Hotel not found... skipping")
    
    

Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping


In [48]:
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
3,Harper,LR,75.15,4.3750,-7.7169,Screensaver
5,Male,MV,80.55,4.1748,73.5089,Hotel Octave Maldives
6,Torbay,CA,77.77,47.6666,-52.7314,Gallows Cove
8,Tahta,EG,88.11,26.7693,31.5021,فندق الزيات
11,Nishihara,JP,83.48,26.1842,127.7558,Minshuku Agaihama
12,Albany,US,78.94,42.6001,-73.9662,
13,Thinadhoo,MV,80.92,0.5333,72.9333,
18,Port-Gentil,GA,76.96,-0.7193,8.7815,Fengshui Residence
20,Atuona,PF,77.09,-9.8000,-139.0333,Hôtel Hiva Oa Hanakee Lodge
35,Eyl,SO,76.48,7.9803,49.8164,Amiin Hotel


In [55]:
# Add a heatmap of temp for the vacation spots

locations = hotel_df[['Lat', 'Lng']]

max_temp = hotel_df['Max Temp']

fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False, max_intensity=300, point_radius=4)

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(heat_layer)

fig.add_layer(marker_layer)

# Call the figure to plot the data

fig

Figure(layout=FigureLayout(height='420px'))

In [54]:
info_box_template = """

<dl>

<dt>Hotel Name</dt><dd>{Hotel Name}</dd>

<dt>City</dt><dd>{City}</dd>

<dt>Country</dt><dd>{Country}</dd>

<dt>Max Temp</dt><dd>{Max Temp} °F</dd>

</dl>

"""

# Store the df Row

hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]